In [1]:
import torch 
import torch.nn as nn 

from EEG_inception_dataset import EEG_inception
from pytorch_lightning import LightningModule,Trainer
import torchmetrics
from torch.utils.data import TensorDataset,DataLoader


### chrononet model

In [5]:
class Block(nn.Module):
    def __init__(self, inplace):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels=inplace, out_channels=32, kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv1d(in_channels=inplace, out_channels=32, kernel_size=4, stride=2, padding=1)
        self.conv3 = nn.Conv1d(in_channels=inplace, out_channels=32, kernel_size=8, stride=2, padding=3)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x1 = self.relu(self.conv1(x))
        x2 = self.relu(self.conv2(x))
        x3 = self.relu(self.conv3(x))
        x = torch.cat([x1, x2, x3], dim=1)
        return x

class ChronoNet(nn.Module):
    def __init__(self, channel):
        super().__init__()
        self.block1 = Block(channel)
        self.block2 = Block(96)
        self.block3 = Block(96)
        
        # Adjust GRU layers to handle the updated input dimensions
        self.gru1 = nn.GRU(input_size=96, hidden_size=32, batch_first=True)
        self.gru2 = nn.GRU(input_size=32, hidden_size=32, batch_first=True)
        self.gru3 = nn.GRU(input_size=64, hidden_size=32, batch_first=True)
        self.gru4 = nn.GRU(input_size=96, hidden_size=32, batch_first=True)
        
        self.gru_linear = nn.Linear(125, 1)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(32, 1)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        # print(x.shape)
        x = x.permute(0, 2, 1)
        
        gru_out1, _ = self.gru1(x)
        gru_out2, _ = self.gru2(gru_out1)
        gru_out = torch.cat([gru_out1, gru_out2], dim=2)
        # print(gru_out.shape, "1")

        gru_out3, _ = self.gru3(gru_out)
        gru_out = torch.cat([gru_out1, gru_out2, gru_out3], dim=2)
        # print(gru_out.shape, "b")
        linear_out = self.relu(self.gru_linear(gru_out.permute(0, 2, 1)))
        gru_out4, _ = self.gru4(linear_out.permute(0, 2, 1))
        # print(gru_out.shape, "c")
        x = self.flatten(gru_out4)
        x = self.fc1(x)
        return x

In [6]:
input=torch.randn(3,8,1000)
input.shape
model=ChronoNet(8)
out=model(input)
out.shape
print(torch.sigmoid(out))


tensor([[0.4591],
        [0.4604],
        [0.4596]], grad_fn=<SigmoidBackward0>)


### instantiating the lighting module for the model 

In [23]:
from pytorch_lightning import LightningModule
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torchmetrics

class ChronoModel(LightningModule):
    def __init__(self):
        super(ChronoModel, self).__init__()
        self.model = ChronoNet(8)
        self.lr = 1e-4
        self.bs = 64
        self.worker = 2
        self.acc = torchmetrics.Accuracy(task="binary")
     
        self.criterion = nn.BCEWithLogitsLoss()

    def forward(self, x):
        return self.model(x)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr)

    def train_dataloader(self):
        dataset = EEG_inception(kind="train", normalize = False)
        return DataLoader(dataset, batch_size=self.bs, num_workers=self.worker, shuffle=True)

    def val_dataloader(self):
        dataset = EEG_inception(kind="val", normalize = False)
        return DataLoader(dataset, batch_size=self.bs, num_workers=self.worker, shuffle=False)

    def training_step(self, batch, batch_idx):
        signal, label = batch
        out = self(signal.float())
        loss = self.criterion(out.flatten(), label.float().flatten())
        preds = (torch.sigmoid(out.flatten()) > 0.5).long()
        # print(preds.)
        acc = self.acc(preds, label.long().flatten())
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log('train_acc', acc, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        signal, label = batch
        out = self(signal.float())
        loss = self.criterion(out.flatten(), label.float().flatten())
        preds = (torch.sigmoid(out.flatten()) > 0.5).long()
        acc = self.acc(preds, label.long().flatten())
        self.log('val_loss', loss, on_epoch=True, prog_bar=True, logger=True)
        self.log('val_acc', acc, on_epoch=True, prog_bar=True, logger=True)
        return loss
    
    def on_train_epoch_end(self):
        # Retrieve the average training loss
        train_loss = self.trainer.callback_metrics.get("train_loss_epoch", None)
        if train_loss is not None:
            self.print(f"Epoch {self.current_epoch} - Training Loss: {train_loss:.4f}")

    def on_validation_epoch_end(self):
        # Retrieve the average validation loss
        val_loss = self.trainer.callback_metrics.get("val_loss", None)
        if val_loss is not None:
            self.print(f"Epoch {self.current_epoch} - Validation Loss: {val_loss:.4f}")


In [24]:
model=ChronoModel()

In [25]:
trainer=Trainer(max_epochs=1000)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [26]:
trainer.fit(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type              | Params | Mode 
--------------------------------------------------------
0 | model     | ChronoNet         | 130 K  | train
1 | acc       | BinaryAccuracy    | 0      | train
2 | criterion | BCEWithLogitsLoss | 0      | train
--------------------------------------------------------
130 K     Trainable params
0         Non-trainable params
130 K     Total params
0.523     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

(800, 8, 1000) in here dataset
val kind 800 800 800
main_job done


c:\Users\zokov\.conda\envs\py3\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:419: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.


Epoch 0 - Validation Loss: 0.6632
(1000, 8, 1000) in here dataset
train kind 1000 1000 1000
main_job done


...

Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 449 - Validation Loss: 0.6909
Epoch 449 - Training Loss: 0.6910


Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 450 - Validation Loss: 0.6906
Epoch 450 - Training Loss: 0.6904


c:\Users\zokov\.conda\envs\py3\Lib\site-packages\pytorch_lightning\trainer\call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


In [ ]:
print(trainer.callback_metrics)

{'train_loss': tensor(0.6919), 'train_loss_step': tensor(0.7212), 'train_acc': tensor(0.5230), 'train_acc_step': tensor(0.3750), 'val_loss': tensor(0.6909), 'val_acc': tensor(0.5275), 'train_loss_epoch': tensor(0.6919), 'train_acc_epoch': tensor(0.5230)}


In [ ]:
train_data = EEG_inception(kind="train", normalize = False)
dl = DataLoader(train_data, batch_size=64, num_workers=2, shuffle=True)

(1000, 8, 1000) in here dataset
train kind 1000 1000 1000
main_job done


In [ ]:
pred_label = []
true_label = []

device = torch.device("cpu")
for ii, (data, label) in enumerate(dl):
    input = data.float().to(device)
    label = label.to(device)

    pred = model(input).float()
    pred = torch.sigmoid(pred)
    pred = (pred >= 0.5).float().to(device).data

    pred = pred.view(-1)
    
    pred_label.append(pred)
    true_label.append(label)
    # print(pred, "val")
    # print(label)
pred_label = torch.cat(pred_label, 0)
true_label = torch.cat(true_label, 0)

val_accuracy = torch.sum(pred_label == true_label).type(torch.FloatTensor) / true_label.size(0)

print(val_accuracy)

tensor(0.5210)


In [ ]:
num_ones = torch.sum(pred_label == 0).item()

In [ ]:
num_ones

795

In [ ]:
torch.sum(pred_label == 1).item()

2559

In [14]:
########################################################################

In [15]:
import torch
import torch.nn as nn

class Block(nn.Module):
    def __init__(self, inplace):
        super().__init__()
        # Add batch normalization and proper initialization
        self.conv1 = nn.Conv1d(in_channels=inplace, out_channels=32, kernel_size=2, stride=2, padding=0)
        self.bn1 = nn.BatchNorm1d(32)
        self.conv2 = nn.Conv1d(in_channels=inplace, out_channels=32, kernel_size=4, stride=2, padding=1)
        self.bn2 = nn.BatchNorm1d(32)
        self.conv3 = nn.Conv1d(in_channels=inplace, out_channels=32, kernel_size=8, stride=2, ====padding=3)
        self.bn3 = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        
        # Initialize weights properly
        nn.init.kaiming_normal_(self.conv1.weight)
        nn.init.kaiming_normal_(self.conv2.weight)
        nn.init.kaiming_normal_(self.conv3.weight)
    
    def forward(self, x):
        x1 = self.dropout(self.relu(self.bn1(self.conv1(x))))
        x2 = self.dropout(self.relu(self.bn2(self.conv2(x))))
        x3 = self.dropout(self.relu(self.bn3(self.conv3(x))))
        x = torch.cat([x1, x2, x3], dim=1)
        return x

class ChronoNet(nn.Module):
    def __init__(self, channel):
        super().__init__()
        self.block1 = Block(channel)
        self.block2 = Block(96)
        self.block3 = Block(96)
        
        # Add dropout and batch normalization to GRU layers
        self.gru1 = nn.GRU(input_size=96, hidden_size=32, batch_first=True, dropout=0.2)
        self.gru2 = nn.GRU(input_size=32, hidden_size=32, batch_first=True, dropout=0.2)
        self.gru3 = nn.GRU(input_size=64, hidden_size=32, batch_first=True, dropout=0.2)
        self.gru4 = nn.GRU(input_size=96, hidden_size=32, batch_first=True, dropout=0.2)
        
        self.gru_linear = nn.Linear(225, 1)
        self.bn_linear = nn.BatchNorm1d(32)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(32, 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        
        # Initialize the linear layers
        nn.init.xavier_normal_(self.gru_linear.weight)
        nn.init.xavier_normal_(self.fc1.weight)
    
    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = x.permute(0, 2, 1)
        
        gru_out1, _ = self.gru1(x)
        gru_out2, _ = self.gru2(gru_out1)
        gru_out = torch.cat([gru_out1, gru_out2], dim=2)
        
        gru_out3, _ = self.gru3(gru_out)
        gru_out = torch.cat([gru_out1, gru_out2, gru_out3], dim=2)
        
        linear_out = self.dropout(self.relu(self.gru_linear(gru_out.permute(0, 2, 1))))
        gru_out4, _ = self.gru4(linear_out.permute(0, 2, 1))
        
        x = self.flatten(gru_out4)
        x = self.bn_linear(x)
        x = self.dropout(x)
        x = self.fc1(x)
        return x

class ChronoModel(LightningModule):
    def __init__(self):
        super(ChronoModel, self).__init__()
        self.model = ChronoNet(8)
        self.lr = 1e-3  # Increased learning rate
        self.bs = 32    # Reduced batch size
        self.worker = 2
        self.acc = torchmetrics.Accuracy(task="binary")
        self.criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([1.0]))  # Balanced weight
        
        # Add early stopping metrics
        self.best_val_acc = 0.0
        self.patience_counter = 0
        
    def forward(self, x):
        return self.model(x)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr, weight_decay=1e-5)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='max', factor=0.1, patience=3, verbose=True
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "monitor": "val_acc"
            }
        }

    def training_step(self, batch, batch_idx):
        signal, label = batch
        out = self(signal.float())
        loss = self.criterion(out.flatten(), label.float().flatten())
        
        # Add gradient clipping
        torch.nn.utils.clip_grad_norm_(self.parameters(), max_norm=1.0)
        
        preds = torch.sigmoid(out.flatten())
        acc = self.acc(preds, label.long().flatten())
        
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log('train_acc', acc, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        
        return loss

    def validation_step(self, batch, batch_idx):
        signal, label = batch
        out = self(signal.float())
        loss = self.criterion(out.flatten(), label.float().flatten())
        
        preds = torch.sigmoid(out.flatten())
        acc = self.acc(preds, label.long().flatten())
        
        self.log('val_loss', loss, on_epoch=True, prog_bar=True, logger=True)
        self.log('val_acc', acc, on_epoch=True, prog_bar=True, logger=True)
        
        return loss

SyntaxError: invalid syntax (818234999.py, line 12)

In [ ]:
# Initialize and train
model = ChronoModel()
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
trainer = Trainer(
    max_epochs=50,
    callbacks=[
        EarlyStopping(monitor='val_loss', patience=5),
        ModelCheckpoint(monitor='val_acc', mode='max')
    ]
)
trainer.fit(model)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
C:\Users\zokov\.conda\envs\py3\Lib\site-packages\pytorch_lightning\trainer\configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
C:\Users\zokov\.conda\envs\py3\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "

  | Name      | Type              | Params | Mode 
--------------------------------------------------------
0 | model     | ChronoNet         | 131 K  | train
1 | acc       | BinaryAccuracy    | 0      | train
2 | criterion | BCEWithLogitsLoss | 0      | train
--------------------------------------------------------
131 K     Trainable params
0         Non-trainable params
131 K     To

MisconfigurationException: `train_dataloader` must be implemented to be used with the Lightning Trainer